In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [2]:
import pandas as pd
import os

df = pd.read_csv("training_metadata.csv")
df = df.dropna(subset=["filename", "sub_category_2"])
df = df[df["filename"].apply(os.path.exists)]

In [ ]:
counts = df["sub_category_2"].value_counts()
print(df["sub_category_2"].nunique())

valid_labels = counts[counts >= 2].index
df = df[df["sub_category_2"].isin(valid_labels)].reset_index(drop=True)

print(df["sub_category_2"].nunique())

445
443


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["sub_category_2"])
num_classes = df["label"].nunique()


In [5]:
import json

label_map = {
    label: int(index)
    for label, index in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))
}

with open("label_map.json", "w") as f:
    json.dump(label_map, f, indent=2)


In [7]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)


In [8]:
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import torch

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class GroceryDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row["filename"]).convert("RGB")
        image = self.transform(image)
        label = row["label"]
        return image, label


In [9]:
from torch.utils.data import DataLoader
from torchvision import models
import torch.nn as nn

train_ds = GroceryDataset(train_df, transform)
val_ds = GroceryDataset(val_df, transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)

# Pretrained ResNet
model = models.resnet18(pretrained=True)
print(model.fc.out_features)
model.fc = nn.Linear(model.fc.in_features, df["label"].max()+1)  # num classes
print(model.fc.out_features)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


d:\tortoise\cv\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\tortoise\cv\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


1000
443


In [10]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(15):  # increase this for real training
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device),  labels.long().to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


d:\tortoise\cv\.venv\Lib\site-packages\PIL\Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1, Loss: 3158.0911
Epoch 2, Loss: 2032.2513
Epoch 3, Loss: 1408.3367
Epoch 4, Loss: 934.0059
Epoch 5, Loss: 594.1356
Epoch 6, Loss: 391.0726
Epoch 7, Loss: 286.7625
Epoch 8, Loss: 232.2887
Epoch 9, Loss: 202.9704
Epoch 10, Loss: 182.2860
Epoch 11, Loss: 170.0679
Epoch 12, Loss: 159.2648
Epoch 13, Loss: 162.0716
Epoch 14, Loss: 142.0301
Epoch 15, Loss: 131.4152


In [16]:
for epoch in range(1):  # increase this for real training
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device),  labels.long().to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 121.8228


In [15]:
torch.save(model.state_dict(), "./assets/grocery_model2.pth")

In [21]:
label_map.keys()

dict_keys(['Adult Diapers', 'Adult Games', 'Aerated, Still, Sparkling', 'Agarbatti, Incense Sticks', 'Air Freshener', 'Almonds', 'Aluminium Foil, Clingwrap', 'Antiseptics & Bandages', 'Apples & Pomegranate', 'Aromatherapy', 'Art Supplies', 'Atta Whole Wheat', 'Attar', 'Ayurveda', 'Baby & Toddler Toys', 'Bagels & Baguette', 'Bakery Biscuits, Cookies', 'Bakeware Accessories', 'Bakeware Moulds, Cutters', 'Baking Accessories', 'Baking Tools & Brushes', 'Baking, Cake Decorations', 'Balsamic & Cider Vinegar', 'Banana, Sapota & Papaya', 'Basmati Rice', 'Bath & Shower', 'Bath Linen', 'Bath Salts & Oils', 'Bath Stool, Basin & Sets', 'Bathing Accessories', 'Bathing Bars & Soaps', 'Beans & Pulses', 'Beans, Brinjals & Okra', 'Bedsheets', 'Birthday & Party Cakes', 'Blankets & Comforters', 'Blended Cooking Oils', 'Blended Masalas', 'Board Games & Puzzles', 'Body Care', 'Body Scrubs & Exfoliants', 'Body Sprays & Mists', 'Boiled & Steam Rice', 'Bowls & Vessels', 'Bread Sticks & Lavash', 'Breadcrumbs &

In [22]:
from sklearn.metrics import classification_report

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        outputs = model(images)
        preds = outputs.argmax(dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

print(classification_report(
    all_labels,
    all_preds,
    target_names=list(label_map.keys()),
    labels=list(label_map.values())
))


d:\tortoise\cv\.venv\Lib\site-packages\PIL\Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


                                   precision    recall  f1-score   support

                    Adult Diapers       1.00      0.73      0.84        11
                      Adult Games       0.00      0.00      0.00         1
        Aerated, Still, Sparkling       0.67      0.50      0.57         8
        Agarbatti, Incense Sticks       0.57      0.40      0.47        10
                    Air Freshener       0.33      0.46      0.39        13
                          Almonds       0.00      0.00      0.00         1
        Aluminium Foil, Clingwrap       0.62      0.42      0.50        12
           Antiseptics & Bandages       0.11      0.29      0.16         7
             Apples & Pomegranate       0.46      0.60      0.52        10
                     Aromatherapy       0.67      0.29      0.40         7
                     Art Supplies       0.30      0.37      0.33        51
                 Atta Whole Wheat       0.00      0.00      0.00         1
                        

d:\tortoise\cv\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\tortoise\cv\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\tortoise\cv\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
d:\tortoise\cv\.venv\Lib\site-packages\sk